# Guest House - Medium

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

········


In [2]:
booking = pd.read_sql_table('booking', engine)
guest = pd.read_sql_table('guest', engine)
room = pd.read_sql_table('room', engine)
rate = pd.read_sql_table('rate', engine)
extra = pd.read_sql_table('extra', engine)

## 6.
Ruth Cadbury. Show the total amount payable by guest Ruth Cadbury for her room bookings. You should JOIN to the rate table using room_type_requested and occupants.

```
+--------------------+
| SUM(nights*amount) |
+--------------------+
|             552.00 |
+--------------------+
```

In [3]:
a = (guest.loc[(guest['first_name']=='Ruth') & 
           (guest['last_name']=='Cadbury'), :]
 .merge(booking, left_on='id', right_on='guest_id')
 .merge(rate, left_on=['room_type_requested', 'occupants'], 
        right_on=['room_type', 'occupancy']))

pd.DataFrame({'SUM(nightsa*amount)': [sum(a['nights']*a['amount'])]})

,SUM(nightsa*amount)
0,552.0


## 7.
Including Extras. Calculate the total bill for booking 5346 including extras.

```
+-------------+
| SUM(amount) |
+-------------+
|      118.56 |
+-------------+
```

In [4]:
a = (booking.loc[booking['booking_id']==5346, :]
     .merge(rate, left_on=['occupants', 'room_type_requested'],
            right_on=['occupancy', 'room_type'])
    [['booking_id', 'amount']])
b = (booking.loc[booking['booking_id']==5346, :]
     .merge(extra, on='booking_id')
    [['booking_id', 'amount']])
c = pd.concat([a, b])

pd.DataFrame({'SUM(amount)': [sum(c['amount'])]})

,SUM(amount)
0,118.56


## 8.
Edinburgh Residents. For every guest who has the word “Edinburgh” in their address show the total number of nights booked. Be sure to include 0 for those guests who have never had a booking. Show last name, first name, address and number of nights. Order by last name then first name.

```
+-----------+------------+---------------------------+--------+
| last_name | first_name | address                   | nights |
+-----------+------------+---------------------------+--------+
| Brock     | Deidre     | Edinburgh North and Leith |      0 |
| Cherry    | Joanna     | Edinburgh South West      |      0 |
| Murray    | Ian        | Edinburgh South           |     13 |
| Sheppard  | Tommy      | Edinburgh East            |      0 |
| Thomson   | Michelle   | Edinburgh West            |      3 |
+-----------+------------+---------------------------+--------+
```

In [5]:
(booking.merge(guest.loc[guest['address'].str.contains('Edinburgh'), :],
               left_on='guest_id', right_on='id', how='right')
 .groupby(['id', 'last_name', 'first_name', 'address'])
 .agg({'nights': 'sum'})
 .reset_index()
 [['last_name', 'first_name', 'address', 'nights']]
 .sort_values(['last_name', 'first_name']))

,last_name,first_name,address,nights
2,Brock,Deidre,Edinburgh North and Leith,0.0
3,Cherry,Joanna,Edinburgh South West,0.0
0,Murray,Ian,Edinburgh South,13.0
4,Sheppard,Tommy,Edinburgh East,0.0
1,Thomson,Michelle,Edinburgh West,3.0


## 9.
How busy are we? For each day of the week beginning 2016-11-25 show the number of bookings starting that day. Be sure to show all the days of the week in the correct order.

```
+------------+----------+
| i          | arrivals |
+------------+----------+
| 2016-11-25 |        7 |
| 2016-11-26 |        8 |
| 2016-11-27 |       12 |
| 2016-11-28 |        7 |
| 2016-11-29 |       13 |
| 2016-11-30 |        6 |
| 2016-12-01 |        7 |
+------------+----------+
```

In [6]:
(booking.loc[(booking['booking_date'] >= '2016-11-25') &
             (booking['booking_date'] <= pd.Timestamp('2016-11-25') + pd.Timedelta(days=6)), :]
 .groupby('booking_date')
 .agg(arrivals=('booking_id', 'count'))
 .reset_index())

,booking_date,arrivals
0,2016-11-25,7
1,2016-11-26,8
2,2016-11-27,12
3,2016-11-28,7
4,2016-11-29,13
5,2016-11-30,6
6,2016-12-01,7


## 10.
How many guests? Show the number of guests in the hotel on the night of 2016-11-21. Include all occupants who checked in that day but not those who checked out.

```
+----------------+
| SUM(occupants) |
+----------------+
|             39 |
+----------------+
```

In [7]:
a = (booking.loc[(booking['booking_date'] <= '2016-11-21') &
             (booking['booking_date'] + 
              pd.to_timedelta(booking['nights'], unit='D') > 
              pd.Timestamp('2016-11-21')), :])
pd.DataFrame({'SUM(occupants)': [sum(a['occupants'])]})

,SUM(occupants)
0,39
